<a href="https://colab.research.google.com/github/mdjamina/m1_ml_lang_detector/blob/main/ML_lang_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Corpus
[Tatoeba](https://tatoeba.org/fr/downloads
) est une collection de phrases et de traductions.

## Téléchargement du corpus

In [ ]:
!wget https://downloads.tatoeba.org/exports/sentences.tar.bz2

--2022-03-09 23:57:27--  https://downloads.tatoeba.org/exports/sentences.tar.bz2
Résolution de downloads.tatoeba.org (downloads.tatoeba.org)… 94.130.77.194
Connexion à downloads.tatoeba.org (downloads.tatoeba.org)|94.130.77.194|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 160000593 (153M) [application/octet-stream]
Enregistre : ‘sentences.tar.bz2’

sentences.tar.bz2   100%[===================>] 152,59M  2,54MB/s    ds 46s     

2022-03-09 23:58:13 (3,33 MB/s) - ‘sentences.tar.bz2’ enregistré [160000593/160000593]



In [ ]:
!wget https://iso639-3.sil.org/sites/iso639-3/files/downloads/iso-639-3-macrolanguages.tab

Extraction de l'archive téléchargée 

In [ ]:
!tar -xf ./sentences.tar.bz2 

## Chargement du corpus

In [ ]:
langs =['eng','pol','deu','fra','spa','ita','tur','por','ara','ckb','rus','ukr']

In [ ]:
import pandas as pd

data = pd.read_csv('./sentences_dev.csv', sep='\t', header=None)
data.columns = ['x','id','I_Id', 'content']
data

,x,id,I_Id,content
0,5481486,5834504,eng,I think we should ask Tom to sing for us.
1,7346165,7747639,ita,Tom voleva dire qualcosa.
2,3870380,4121281,tur,Tom seni benim kadar iyi tanımıyor.
3,908111,952134,pol,"Nie podoba mi się, jak się z niej wyśmiewasz."
4,2561541,2716415,por,Talvez em outra oportunidade!
...,...,...,...,...
1025215,1332637,1410208,fra,Je ne l'ai jamais rencontrée.
1025216,7337354,7738722,kab,Ṭṭfent-asen-tent-id.
1025217,7120657,7513990,eng,I know Tom isn't going to want to do that.
1025218,1403948,1485378,cmn,你是不會說話嗎?


## Nettoyage des données

In [ ]:
#identification des colonnes qui contiennent des valeurs NAN (non renseignées)
data.isnull().any()

id         False
I_Id       False
content    False
dtype: bool

In [ ]:
#suppression des valeurs NAN
data.dropna(subset = ["I_Id"], inplace=True)


## regrouppement des dialects par langue principale

In [ ]:
#TODO
# site iso langues avec leur regroupement
# telecharger fichier csv(iso) contient : langue : arb=dz, ara=egy ..,  le telecharger avec  pandas data frames 

In [ ]:
macro_lang = pd.read_csv('/content/iso-639-3-macrolanguages.tab', sep='\t', header=None)
macro_lang.columns = ['lang',	'I_Id',	'I_Status']

macro_lang

,lang,I_Id,I_Status
0,M_Id,I_Id,I_Status
1,aka,fat,A
2,aka,twi,A
3,ara,aao,A
4,ara,abh,A
...,...,...,...
450,zho,NaN,A
451,zho,wuu,A
452,zho,yue,A
453,zza,diq,A


# pre-processing

In [ ]:
data_counts = pd.DataFrame( data['lang'].value_counts(), columns=['lang','count'] )

In [ ]:
data_counts[data_counts['lang']<100]

## TODO

In [ ]:
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(data['content'], data['I_Id'], test_size=0.33, random_state=1) 
# ramdom_state = meme corpus (meme decoupage, decouper en 2 train et test le corpus, car on a besoin de faire les tests)

In [ ]:
y_train.value_counts()

eng    1052272
rus     604660
ita     537404
tur     480100
epo     457019
        ...   
mnr          1
pli          1
lou          1
gom          1
cyo          1
Name: I_Id, Length: 400, dtype: int64

# Model

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer

model = Pipeline([('TfidfVectorizer', TfidfVectorizer()),
                     ('LinearSVC', LinearSVC()),],verbose=True)


model.fit(x_train, y_train)  




[Pipeline] ... (step 1 of 2) Processing TfidfVectorizer, total=   4.2s
[Pipeline] ......... (step 2 of 2) Processing LinearSVC, total= 9.0min


Pipeline(steps=[('TfidfVectorizer', TfidfVectorizer()),
                ('LinearSVC', LinearSVC())],
         verbose=True)

In [ ]:
predictions = model.predict(x_test)

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[ 0  0  0 ...  0  0  1]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ... 95  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0 16]]


In [ ]:
print(metrics.classification_report(y_test,predictions))

/home/amina/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/amina/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/amina/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/amina/.local/lib/python3.9/site-packages/skle

              precision    recall  f1-score   support

          \N       0.00      0.00      0.00         8
         abk       0.00      0.00      0.00         2
         acm       0.00      0.00      0.00         1
         ady       0.00      0.00      0.00         1
         afb       0.00      0.00      0.00         4
         afh       0.00      0.00      0.00         2
         afr       0.89      0.68      0.77       128
         ain       1.00      0.33      0.50        12
         akl       0.00      0.00      0.00         2
         aln       0.00      0.00      0.00         2
         amh       1.00      0.30      0.46        10
         ang       0.67      0.27      0.38        15
         aoz       0.00      0.00      0.00         2
         apc       1.00      0.33      0.50         3
         ara       0.95      0.92      0.94      1301
         arg       0.50      0.17      0.25         6
         arq       0.75      0.55      0.63        66
         ary       0.00    

/home/amina/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/amina/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
metrics.accuracy_score(y_test,predictions)

0.948265262087597

In [ ]:
model.predict(['그러나 주로 개인적인 용도로 이용되는 위키도 있는데, 이를 개인 위키라고 한다.'])[0] 

'kor'

In [ ]:
model.predict(['ויקי יכולה להיות שיטה טובה לשיתוף ידע בקהילות שפועלות באמצעות האינטרנט או בתוך חברות מסחריות. היא חלופה'])[0]

'heb'

In [ ]:
model.predict(['einsetzt. Zudem nutzen auch viele '])[0]

'deu'

In [ ]:
model.predict(['এটা নতুনকৈ সৃষ্টি কৰিব পাৰি বা ইতিপূৰ্বে থকা পৃষ্ঠা এটা সম্পাদনা কৰিব পাৰি'])[0]

'asm'

In [ ]:


model.predict(['से प्रत्येक एक विशिष्ट भाषा से संबंधित है। विकिपीडिया के अलावा, सार्वजनिक और निजी दोनों उपयोग में सैकड़ों हजारों अ'])[0]

'mar'

# Persistence

In [ ]:
from sklearn.externals import joblib
joblib.dump(model, 'lang_detector.pkl') 